In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 51.2 MB/s eta 0:00:00


In [ ]:
!pip install tqdm

In [ ]:
!pip install pmapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.5/567.5 kB 7.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd

select = pd.read_csv("selected_ligands3.smi", sep='\t', header=None)
select.rename(columns={0: 'smi', 1: 'ID'}, inplace=True)

df_smi_id = select[['ID', 'smi']]


df_smi_id.to_csv("smi_id_only.smi", index=False)

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm import tqdm
from pmapper.pharmacophore import Pharmacophore as P
from multiprocessing import Pool, cpu_count

# --------------------------------
# 1. 입력 및 데이터 정리
# --------------------------------
# 쉼표(,) 구분자와 'ID', 'smi' 열 이름을 사용합니다.
df = pd.read_csv("smi_id_only.smi", sep=",", names=['ID','smi'])

# TypeError 방지를 위한 데이터 정리 및 타입 변환 (필수)
df.dropna(subset=['smi'], inplace=True)
df['smi'] = df['smi'].astype(str)

# 병렬 처리를 위해 데이터프레임을 리스트로 변환
data_to_process = df[['ID', 'smi']].to_dict('records')

print(f"✅ 데이터 준비 완료. 총 {len(data_to_process)}개 분자를 {cpu_count()}개 코어로 처리합니다.")

# --------------------------------
# 2. 병렬 처리를 위한 단일 작업 함수 (Worker Function)
# --------------------------------
def process_molecule(row):
    """단일 분자에 대해 3D 임베딩 및 Pharmacophore Signature를 계산합니다."""
    mol_id = row['ID']
    smi = row['smi']

    # 입력 유효성 검사 및 문자열 변환
    smi = str(smi).strip()
    if not smi or smi.lower() in ('nan', 'none'):
        return mol_id, None

    mol = Chem.MolFromSmiles(smi)
    if not mol:
        return mol_id, None

    mol = Chem.AddHs(mol)
    try:
        # 3D 임베딩 (강건성 강화 로직)
        AllChem.EmbedMultipleConfs(mol, numConfs=5, maxAttempts=500, pruneRmsThresh=-1, randomSeed=42)
        # 생성된 Confomer 중 가장 에너지가 낮은 하나를 선택하여 최적화
        AllChem.UFFOptimizeMoleculeConfs(mol)
    except:
        return mol_id, None # 임베딩 실패

    # Pharmacophore signature 생성
    try:
        p = P()
        p.load_from_mol(mol)
        sig = p.get_signature_md5(tol=5)
        return mol_id, sig
    except:
        return mol_id, None # Pharmacophore 계산 실패

# --------------------------------
# 3. 메인 병렬 루프
# --------------------------------
# CPU 코어 수만큼 프로세스를 생성합니다.
num_cores = cpu_count()
print(f"사용 가능한 CPU 코어 수: {num_cores}개")

with Pool(num_cores) as pool:
    # pool.imap_unordered를 사용하여 비동기적으로 결과를 얻고 tqdm으로 진행 상태를 표시합니다.
    results = list(tqdm(pool.imap_unordered(process_molecule, data_to_process),
                        total=len(data_to_process),
                        desc="Parallel Processing"))

# 결과 처리 (ID를 인덱스로 사용하여 Signature를 매핑)
sig_map = {res[0]: res[1] for res in results}
df['pharma_sig'] = df['ID'].map(sig_map)

# --------------------------------
# 4. 저장
# --------------------------------
df[['ID','smi','pharma_sig']].to_csv("pharma_sig_output_parallel.csv", index=False)
print("Done! ✅ 병렬 처리로 계산 및 'pharma_sig_output_parallel.csv' 파일로 저장 완료!")



✅ 데이터 준비 완료. 총 30260개 분자를 2개 코어로 처리합니다.
사용 가능한 CPU 코어 수: 2개


[23:59:43] SMILES Parse Error: syntax error while parsing: smi
[23:59:43] SMILES Parse Error: check for mistakes around position 2:
Parallel Processing:   0%|          | 0/30260 [00:00<?, ?it/s][23:59:43] smi
[23:59:43] ~^
[23:59:43] SMILES Parse Error: Failed parsing SMILES 'smi' for input: 'smi'
Parallel Processing:   0%|          | 13/30260 [00:08<5:43:25,  1.47it/s]


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np

# --- 설정 및 파일 매핑 확인 ---
FP_SCORE_FILE = "selected_ligands3.smi"
PHARMA_SIG_FILE = "pharma_sig_output_parallel.csv"
OUTPUT_MERGED_FILE = "merged_data_for_selection.csv"

# 분석을 통해 확인된 컬럼 이름
ID_COLUMN = 'ID'
SMILES_COLUMN = 'smi'
# 'selected_ligands3.smi'의 4번째 컬럼에 있는 Tanimoto 점수를 사용합니다.
SIMILARITY_COLUMN = 'Score_FP_main'

# --------------------------------
# 1. FP Score 데이터 로드 (selected_ligands3.smi)
# --------------------------------
# 탭 구분자로 로드하며, 헤더가 없으므로 컬럼 이름을 직접 지정합니다.
df_fp = pd.read_csv(
    FP_SCORE_FILE, sep='\t', header=None,
    names=[SMILES_COLUMN, ID_COLUMN, 'Mol_Object_Garbage', SIMILARITY_COLUMN, 'Score_2', 'Score_3', 'Score_4']
)
# FP 점수 컬럼을 실수형(float)으로 변환합니다.
df_fp[SIMILARITY_COLUMN] = pd.to_numeric(df_fp[SIMILARITY_COLUMN], errors='coerce')
# 최종 병합에 필요한 컬럼만 선택합니다.
df_fp = df_fp[[ID_COLUMN, SMILES_COLUMN, SIMILARITY_COLUMN]]


# --------------------------------
# 2. Pharmacophore Signature 데이터 로드 (pharma_sig_output_parallel.csv)
# --------------------------------
df_pharma = pd.read_csv(PHARMA_SIG_FILE)
# 중복 헤더 행 제거 ('ID' 값이 'ID'인 행)
df_pharma_clean = df_pharma[df_pharma[ID_COLUMN] != ID_COLUMN].copy()
# 시그니처 값이 없는 (계산 실패) 행은 제거합니다.
df_pharma_clean = df_pharma_clean.dropna(subset=['pharma_sig'])


# --------------------------------
# 3. ID를 기준으로 두 데이터 병합 (sig 추가)
# --------------------------------
# 두 데이터프레임을 ID를 기준으로 inner join하여 합칩니다.
df_final = pd.merge(
    df_fp,
    df_pharma_clean[[ID_COLUMN, 'pharma_sig']],
    on=ID_COLUMN,
    how='inner'
)
df_final.dropna(subset=[SIMILARITY_COLUMN], inplace=True)

print(f"✅ 데이터 병합 완료. 최종 대상 분자 수: {len(df_final)}개")
print(f"⭐ 병합된 데이터가 '{OUTPUT_MERGED_FILE}' 파일로 저장되었습니다.")
df_final.to_csv(OUTPUT_MERGED_FILE, index=False)

✅ 데이터 병합 완료. 최종 대상 분자 수: 28574개
⭐ 병합된 데이터가 'merged_data_for_selection.csv' 파일로 저장되었습니다.


In [ ]:
import pandas as pd
import numpy as np

# --- 설정 및 파일 매핑 확인 ---
FP_SCORE_FILE = "selected_ligands3.smi"
PHARMA_SIG_FILE = "pharma_sig_output_parallel.csv"
# ⭐ 최종 파일명을 .smi로 지정
OUTPUT_MERGED_FILE = "merged_data_for_selection_smi_id.smi"
DELIMITER = '\t' # 탭 구분자

# 분석을 통해 확인된 컬럼 이름
ID_COLUMN = 'ID'
SMILES_COLUMN = 'smi'
SIMILARITY_COLUMN = 'Score_FP_main'

# --------------------------------
# 1. FP Score 데이터 로드 및 정리 (selected_ligands3.smi)
# --------------------------------
df_fp = pd.read_csv(
    FP_SCORE_FILE, sep=DELIMITER, header=None,
    names=[SMILES_COLUMN, ID_COLUMN, 'Mol_Object_Garbage', SIMILARITY_COLUMN, 'Score_2', 'Score_3', 'Score_4']
)
df_fp[SIMILARITY_COLUMN] = pd.to_numeric(df_fp[SIMILARITY_COLUMN], errors='coerce')
df_fp = df_fp[[ID_COLUMN, SMILES_COLUMN, SIMILARITY_COLUMN, 'Score_2', 'Score_3', 'Score_4']]


# --------------------------------
# 2. Pharmacophore Signature 데이터 로드 및 정리
# --------------------------------
df_pharma = pd.read_csv(PHARMA_SIG_FILE)
df_pharma_clean = df_pharma[df_pharma[ID_COLUMN] != ID_COLUMN].copy()
df_pharma_clean = df_pharma_clean.dropna(subset=['pharma_sig'])


# --------------------------------
# 3. ID를 기준으로 두 데이터 병합 및 컬럼 순서 조정
# --------------------------------
df_final = pd.merge(
    df_fp,
    df_pharma_clean[[ID_COLUMN, 'pharma_sig']],
    on=ID_COLUMN,
    how='inner'
)
df_final.dropna(subset=[SIMILARITY_COLUMN], inplace=True)

# ⭐ 핵심: 컬럼 순서를 [smi, ID, ...]로 변경
# 나머지 컬럼들은 순서대로 뒤에 붙습니다.
ordered_cols = [SMILES_COLUMN, ID_COLUMN] + [col for col in df_final.columns if col not in [SMILES_COLUMN, ID_COLUMN]]
df_final = df_final[ordered_cols]

print(f"✅ 데이터 병합 완료. 최종 대상 분자 수: {len(df_final)}개")

# 탭 구분자로 파일 저장
df_final.to_csv(OUTPUT_MERGED_FILE, sep=DELIMITER, index=False, header=True)
print(f"⭐ 탭 구분자로 '{OUTPUT_MERGED_FILE}' 파일에 smi, ID 형태로 저장 완료!")

✅ 데이터 병합 완료. 최종 대상 분자 수: 28574개
⭐ 탭 구분자로 'merged_data_for_selection_smi_id.smi' 파일에 smi, ID 형태로 저장 완료!


In [ ]:
# --------------------------------
# 4. FP 점수 (Score_FP_main)로 상위 70% (7,000개) 추출
# --------------------------------
TARGET_COUNT = 7060
OUTPUT_7K_FILE = "top_7000_similarity_selection_smi_id.smi"

# FP 점수(Score_FP_main)를 기준으로 내림차순 정렬
df_sorted = df_final.sort_values(by=SIMILARITY_COLUMN, ascending=False)

# 상위 7,000개 분자 선택
df_top_7k = df_sorted.head(TARGET_COUNT)

# 탭 구분자로 파일 저장 (SMI ID 형태 유지)
df_top_7k.to_csv(OUTPUT_7K_FILE, sep=DELIMITER, index=False, header=True)

print("-" * 30)
print(f"✅ FP 유사성 기준 상위 {len(df_top_7k)}개 후보가 '{OUTPUT_7K_FILE}' 파일에 저장 완료!")


------------------------------
✅ FP 유사성 기준 상위 7060개 후보가 'top_7000_similarity_selection_smi_id.smi' 파일에 저장 완료!


In [ ]:
# --------------------------------
# 3. FP 점수 (Score_FP_main)로 상위 70% (7,000개) 추출
# --------------------------------
TARGET_COUNT = 7060
OUTPUT_7K_FILE = "top_7000_similarity_selection_smi_id_only.smi"

# FP 점수(Score_FP_main)를 기준으로 내림차순 정렬
df_sorted = df_final.sort_values(by=SIMILARITY_COLUMN, ascending=False)

# 상위 7,000개 분자 선택
df_top_7k = df_sorted.head(TARGET_COUNT)

# ⭐ SMI ID 만 파일로 저장 (헤더 없이)
df_top_7k[[SMILES_COLUMN, ID_COLUMN]].to_csv(OUTPUT_7K_FILE, sep=DELIMITER, index=False, header=False)

print("-" * 30)
print(f"✅ FP 유사성 기준 상위 {len(df_top_7k)}개 후보가 SMI ID 형태로 '{OUTPUT_7K_FILE}' 파일에 저장 완료!")

------------------------------
✅ FP 유사성 기준 상위 7060개 후보가 SMI ID 형태로 'top_7000_similarity_selection_smi_id_only.smi' 파일에 저장 완료!
